In [257]:
from datetime import datetime
from dataclasses import dataclass, field

@dataclass
class Library:
    """Creates the operations of the library"""

    #def __init__(self) -> None:

    collection: dict = field(default_factory=dict)
    registrated: dict = field(default_factory=dict)

    def registrate(self, student: Student):
        """Student registration"""

        # Creates the dictionary about the student data for the library
        # Dict -> Name: Class, Balance, Loan informations
        student = student.__dict__

        self.registrated[student["name"]] = {
            "class": student["year"],
            "balance": student["balance"],
            "loaned_books": student["loaned_books"],
        }

        print(
            "{} from class of {} is registrated".format(
                student["name"], student["year"]
            )
        )
        print("Library balance: ${}".format(student["balance"]))

    def add_book(self, book: str, info: dict) -> None:
        """Add books to the library"""

        self.collection[book] = info

    def remove_book(self, book: str) -> None:
        """Remove books from the library"""

        del self.collection[book]

@dataclass
class Student:
    """Creates the actions of the students"""

    name: str
    year: int 
    info: Library
    loaned_books: dict = field(default_factory=dict)
    balance: int = 0

    def __str__(self) -> str:
        """Displays the actual information about the student"""
        
        return f"Name: {self.name}\nClass: {self.year}\nBalance: ${self.balance}\nLoaned books: {self.loaned_books}"

    def deposit_money(self, money):
        """Student deposit on the account"""

        self.balance += money

        return f"Your balance: ${self.balance}"

    def loan_book(self, books: list, date: str):
        """Loaning process"""
        """Argument "books" type is list if the loan request contains several books """

        if self.name not in self.info.registrated or self.balance < 0:

            return f"{self.name} is not registrated in the system yet or your balance is negative. Please contact with the reception"

        elif len(books) > 10:

            return f"Maximum number of loan request is 10!\n Your request number: {len(books)}"

        for book in books:

            loaned = []  # collect the available book
            if book in self.info.collection:

                loaned.append(book)  # add the available book to the list

                # register the loan details(book and date) the student side
                self.loaned_books[book] = self.info.collection[book]
                self.loaned_books[book]["loan_date"] = datetime.strptime(date,"%Y-%m-%d") 
                # register the loan details(book and date) the library side
                self.info.registrated[self.name]["loaned_books"][book] = self.info.collection[book]
                self.info.registrated[self.name]["loaned_books"][book][
                    "loan_date"
                ] = datetime.strptime(date,"%Y-%m-%d")

                # remove the book temporarely from the library
                self.info.remove_book(book)
            else:
                return f"The book {book} is not available!"

        return f"Successfully loaned:{loaned}"

    def return_book(self, books: list, date: str):  # argument "books"
        """Book returning process"""
        """Argument "books" type is list if the returning process contains several books"""

        for book in books:

            if book in self.info.registrated[self.name]["loaned_books"]:

                loan_date = self.info.registrated[self.name]["loaned_books"][book]["loan_date"]   # start date of the loan
                return_date = datetime.strptime(date, "%Y-%m-%d")  # return date

                diff = (return_date - loan_date).days # difference between start and return date

                # remove the loan_date element from the loaned book
                self.info.registrated[self.name]["loaned_books"][book].pop("loan_date")

                # book get back to the library
                self.info.add_book(book, self.info.registrated[self.name]["loaned_books"][book])

                # loaned book and its loan date is getting deleted from the system
                self.info.registrated[self.name]["loaned_books"].pop(book)

                #print(self.loaned_books)
                #print(self.info.registrated[self.name]["loaned_books"])
#
                #self.loaned_books.pop(book)
#
                # if the loaning time exceeds the 30 day rule
                if diff > 30:

                    self.balance -= diff
                    return f"Due to the exceeding of the 30 return limit your balance is charged with ${diff}!" 

            else:
                return f"The book {book} has not been loaned yet!"

        return f"Return is successfully processed!"

In [258]:
library = Library()
tom = Student("Tom Jerry", 2023, library)

library.registrate(tom)

Tom Jerry from class of 2023 is registrated
Library balance: $0


In [259]:
library.registrated

{'Tom Jerry': {'class': 2023, 'balance': 0, 'loaned_books': {}}}

In [260]:
library.add_book("Harry Potter", {"date":2001,"genre":"fantasy"})

In [261]:
library.collection

{'Harry Potter': {'date': 2001, 'genre': 'fantasy'}}

In [262]:
tom.loan_book(["Harry Potter"], "2023-01-24")

"Successfully loaned:['Harry Potter']"

In [263]:
'Tom Jerry' in library.registrated

True

In [264]:
tom.info.registrated

{'Tom Jerry': {'class': 2023,
  'balance': 0,
  'loaned_books': {'Harry Potter': {'date': 2001,
    'genre': 'fantasy',
    'loan_date': datetime.datetime(2023, 1, 24, 0, 0)}}}}

In [265]:
print(tom)

Name: Tom Jerry
Class: 2023
Balance: $0
Loaned books: {'Harry Potter': {'date': 2001, 'genre': 'fantasy', 'loan_date': datetime.datetime(2023, 1, 24, 0, 0)}}


In [267]:
tom.return_book(["Harry Potter"], "2023-03-24")

'Due to the exceeding of the 30 return limit your balance is charged with $59!'

In [190]:
library.registrated

{'Tom Jerry': {'class': 2023, 'balance': 0, 'loaned_books': {}}}

In [191]:
library.collection

{'Harry Potter': {'date': 2001, 'genre': 'fantasy'}}

In [268]:
print(tom)

Name: Tom Jerry
Class: 2023
Balance: $-59
Loaned books: {}


In [269]:
tom.loan_book(["Harry Potter"], "2023-01-25")

'Tom Jerry is not registrated in the system yet or your balance is negative. Please contact with the reception'